In [42]:
import imutils
import argparse
import datetime
import time
import cv2
import numpy as np

vs = cv2.VideoCapture("/home/andrea/Desktop/airplane_videos/DSC_2530.MOV")
firstFrame = None

length = 10 #Length, in meters, of the captured frame
upper_limit = 0
lower_limit = 870
frame_rate = 60
dt = 1/frame_rate
starting_height = 0



video_size = None
pxl_width = 0
m_per_px = 0


while True:
    ret, firstFrame = vs.read()
    cv2.imshow("Original frame", firstFrame)
    if (pxl_width == 0):
        video_size = firstFrame.shape
        pxl_width = firstFrame.shape[1]
        m_per_px = length/pxl_width
    firstFrame = cv2.cvtColor(firstFrame, cv2.COLOR_BGR2GRAY)
    firstFrame = cv2.GaussianBlur(firstFrame, (21, 21), 0) #(21, 21), 0
    cv2.imshow("First frame", firstFrame)
    key = cv2.waitKey(15) & 0xFF
    # if the `q` key is pressed, break from the loop
    if key == ord("q"):
        break

print("Length", length)
print("pxl_width", pxl_width)
print("m_per_px", m_per_px)
print("video_size", video_size)
print("dt", dt)

airplane_contours = []
should_update = True

frame = None

while True:
    cv2.imshow("First frame", firstFrame)
    
    # grab the current frame and initialize the occupied/unoccupied
    # text
    if should_update:
        ret, frame = vs.read()
    else:
        text = "Multiple"
        
    if not ret:
        break
        

    text = "Not Found"

    # resize the frame, convert it to grayscale, and blur it
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    
    # compute the absolute difference between the current frame and
    # first frame
    frameDelta = cv2.absdiff(firstFrame, gray)
#     frameDelta = np.sqrt(np.array(frameDelta))
    thresh = cv2.threshold(frameDelta, 15, 255, cv2.THRESH_BINARY)[1]
    # dilate the thresholded image to fill in holes, then find contours
    # on thresholded image
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    # loop over the contours
    
    cv2.line(frame, (0, upper_limit), (video_size[1]-1, upper_limit), (0, 255, 0), thickness=2)
    cv2.line(frame, (0, lower_limit), (video_size[1]-1, lower_limit), (0, 255, 0), thickness=2)
    
    valid_contours = 0;
    
#     possible_rects = []
    
    for c in cnts:
        
        if not should_update:
            break
        # if the contour is too small, ignore it
        if cv2.contourArea(c) < 250:
            continue
        
        # compute the bounding box for the contour, draw it on the frame,
        # and update the text
        rect = cv2.boundingRect(c)
        (x, y, w, h) = rect
        
        if y < upper_limit:
            continue
            
        if y > lower_limit:
            continue
        
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        text = "Found"
        airplane_contours.append((x, y, w, h))
#         possible_rects.append((x, y, w, h))
        valid_contours += 1
        image = cv2.circle(frame, (rect[0]+rect[2],int(rect[1]+rect[3]/2)), radius=5, color=(0, 0, 255), thickness=-5)
        
#     if len(possible_rects) == 1:
#         airplane_contours.append(possible_rects[0])
#     else:
#         closest = 
    
    if valid_contours > 1:
        should_update = False
    
    # draw the text and timestamp on the frame
    cv2.putText(frame, "Paper Airplane Status: {}".format(text), (10, 20),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    
    # show the frame and record if the user presses a key
    cv2.imshow("Video Feed", frame)
    cv2.imshow("Thresh", thresh)
    cv2.imshow("Frame Delta", frameDelta)
    key = cv2.waitKey(15) & 0xFF
    # if the `q` key is pressed, break from the loop
    if key == ord("q"):
        break

# print(airplane_contours)
print(len(airplane_contours))

centerpoints_px = []

for rect in airplane_contours:
    centerpoints_px.append((rect[0]+rect[2], video_size[0] - (rect[1]+rect[2]/2) ))
        
centerpoints_m = []
t = 0
height_diff = centerpoints_px[0][1]*m_per_px - starting_height
for pt in centerpoints_px:
    centerpoints_m.append((t, pt[0]*m_per_px, pt[1]*m_per_px - height_diff))
    t+=dt
    
print(centerpoints_m)

a = np.array(centerpoints_m)

vs.release()
cv2.destroyAllWindows()

Length 10
pxl_width 1920
m_per_px 0.005208333333333333
video_size (1080, 1920, 3)
dt 0.016666666666666666
113
[(0, 0.43229166666666663, 0.0), (0.016666666666666666, 0.5572916666666666, 0.02864583333333348), (0.03333333333333333, 0.6822916666666666, 0.06510416666666696), (0.05, 0.7916666666666666, 0.1171875), (0.06666666666666667, 0.9010416666666666, 0.171875), (0.08333333333333333, 1.0, 0.23177083333333348), (0.09999999999999999, 1.09375, 0.296875), (0.11666666666666665, 1.1822916666666665, 0.3645833333333335), (0.13333333333333333, 1.2604166666666665, 0.4348958333333335), (0.15, 1.328125, 0.5078125), (0.16666666666666666, 1.3958333333333333, 0.578125), (0.18333333333333332, 1.453125, 0.6484375), (0.19999999999999998, 1.5052083333333333, 0.7083333333333335), (0.21666666666666665, 1.5572916666666665, 0.7734375), (0.2333333333333333, 1.6041666666666665, 0.8333333333333335), (0.24999999999999997, 1.6458333333333333, 0.8880208333333335), (0.26666666666666666, 1.6875, 0.9427083333333335), (

In [43]:
import pandas as pd
import numpy as np

a = np.array(centerpoints_m)

df = pd.DataFrame(a)

df

,0,1,2
0,0.000000,0.432292,0.000000
1,0.016667,0.557292,0.028646
2,0.033333,0.682292,0.065104
3,0.050000,0.791667,0.117188
4,0.066667,0.901042,0.171875
...,...,...,...
108,1.800000,2.291667,-0.721354
109,1.816667,2.291667,-0.721354
110,1.833333,2.291667,-0.718750
111,1.850000,2.291667,-0.721354


In [17]:
dir_name = input("File directory: ") #Run this once!!

File directory: /home/andrea/Desktop/airplane_videos


In [40]:
file_name = input("File name: ")

File name: 10-50_ailerons_parallel_2.csv


In [44]:
df.to_csv(dir_name + "/" + file_name)